In [7]:
import cvxpy as cp
import numpy as np

from basis_generator import basis_gs, sel_seq

# Parameters

In [57]:
dimX = 3
dim_base = 2
num_obs = 2
len_seq = 3
num_out = 2
remove_last_out = False

GYN_events = [((0, 0, 0), (0, 0, 0)),
            ((1, 1, 0), (0, 1, 1)),
            ((0, 1, 1), (1, 0, 1)),
            ((1, 0, 1), (1, 1, 0)),]

In [58]:
sequences = sel_seq(num_obs = num_obs,
                len_seq = len_seq,
                num_out = num_out,
                remove_last_out = False)

C = np.zeros(X_basis[0].shape)
for i, seq_row in enumerate(sequences):
    if seq_row in GYN_events:
        C[i,i] = 1

# Optimization with the First Level of the Hierarchy

In [59]:
X_basis = basis_gs(dim = dim_base,
            num_obs = num_obs,
            len_seq = len_seq,
            num_out = num_out,
            level = 1,
            remove_last_out = remove_last_out,
            stop = 1000)

(64, 64)
Nul matrix found
Number of LI moment matrices:  13


In [60]:
alpha = cp.Variable((len(X_basis), 1))
X = cp.Variable(X_basis[0].shape, symmetric=True)

constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
for i in range(8):
    constraints += [
        sum([X[i+k*8,i+k*8] for k in range(8)]) == 1
    ]   

In [61]:
prob = cp.Problem(cp.Maximize(cp.trace(C@X)),
                  constraints)

In [62]:
prob.solve(solver=cp.MOSEK, verbose=True)

print(prob.status)
print("The optimal value is", prob.value)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 12296           
  Cones                  : 0               
  Scalar variables       : 2093            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 12296           
  Cones                  : 0               
  Scalar variables       : 2093            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 12              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4596
Optimizer  - Cones                  : 1
Optimizer  - Scalar variable